In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
import numpy as np
from utilities import Tiempo
import dataExtraction
import statistics

In [ ]:
#Si tenerEnCuentaMove = False -> cuando se realiza una accion de tipo "move" no se tiene en cuenta para calcular el tiempo entre interacciones, como si no existiese la traza
def extraerTiemposEntreInteracciones(rawData, tenerEnCuentaMove = False):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        timestamp = e["timestamp"]
        
        levelCode = erIdLevel.split(objectId)[-1]
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        if name in ultimaInteraccionJugador:
                            if e["result"]["extensions"]["action"] != "move" or tenerEnCuentaMove: #<-- Que accion realiza
                                t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                                #Añadimos la dif de tiempo al diccionario del jugador

                                if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                    if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "interaccionTarjeta"})
                                    else:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "interaccionTarjeta"}]

                                if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                    if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                                    else:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]

                                ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccionTarjeta"}
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    return {"tiempoInicio_Interaccion_Jugador" : tiempoInicio_Interaccion_Jugador, "tiempoInteraccion_Completado_Jugador" : tiempoInteraccion_Completado_Jugador, "tiempoInteraccion_Interaccion_Jugador" : tiempoInteraccion_Interaccion_Jugador,"anomalias" : anomalias}

In [ ]:
def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in tiempos[name]:
                        tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                    else:
                        tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                else:
                    pass #Aqui entraria si se reinicia el nivel
            elif erCompleted.search(verb):
                if evento["result"]["score"]["raw"] > 0 :
                    if levelCode in tiempos[name]:
                        tiempos[name][levelCode][-1]["fin"] = timestamp
                        tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                        
                elif evento["result"]["score"]["raw"] == -1:
                    if levelCode in tiempos[name]:
                        tiempos[name][levelCode][-1]["fin"] = timestamp
                        tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
    
    return tiempos

In [ ]:
def get_Tiempos_Ordenados(tiemposInteracciones):
    tiempos = defaultdict()
    
    for name in tiemposInteracciones:
        for level in tiemposInteracciones[name]:
            for t in tiemposInteracciones[name][level]:
                if level in tiempos:
                    tiempos[level].append(t)
                else:
                    tiempos[level] = [t]
                    
    for t in tiempos:
        tiempos[t].sort(key=lambda traza: traza["tiempo"])
        
    return tiempos

In [ ]:
#Devuelve un diccionario con clave nivel y valor el tiempo a partir del cual se considera outlier
def getLimiteOutliers(tiemposNivel):
    limiteOutliers = defaultdict()
    
    for level in tiemposNivel:
        valores = [x["tiempo"] for x in tiemposNivel[level]] #Pasamos los timestamps a un array para calcular los cuartiles
        q1 = np.quantile(valores, 0.25) #Primer cuartil
        q3 = np.quantile(valores, 0.75) #Tercer cuartil
        limiteOutliers[level] = ((q3-q1)*1.5)+q3 #Tiempo donde empieza a considerarse outlier

    return limiteOutliers

In [ ]:
def getJugadoresOutliers(limiteOutliers, tiemposInteracciones):
    outliersPorNivel = defaultdict(defaultdict)
    
    for name in tiemposInteracciones:
        for level in tiemposInteracciones[name]:
            for t in tiemposInteracciones[name][level]:
                if t["tiempo"] >= limiteOutliers[level]:
                    if level in outliersPorNivel[name]:
                        outliersPorNivel[name][level].append(t)
                    else:
                        outliersPorNivel[name][level] = [t]
                        
    return outliersPorNivel

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)

tenerEnCuentaMove = False
result = extraerTiemposEntreInteracciones(rawData, tenerEnCuentaMove)

tiempos_Completar_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

JSONFile.close()

In [ ]:
tiempos_Inicio_Interaccion = get_Tiempos_Ordenados(result["tiempoInicio_Interaccion_Jugador"])
tiempos_Interaccion_Completado = get_Tiempos_Ordenados(result["tiempoInteraccion_Completado_Jugador"])
tiempos_Interaccion_Interaccion = get_Tiempos_Ordenados(result["tiempoInteraccion_Interaccion_Jugador"])

outliers_Inicio_Interaccion = getLimiteOutliers(tiempos_Inicio_Interaccion)
outliers_Interaccion_Completado = getLimiteOutliers(tiempos_Interaccion_Completado)
outliers_Interaccion_Interaccion = getLimiteOutliers(tiempos_Interaccion_Interaccion)

In [ ]:
#Tiempo entre que inicia el nivel y realiza la primera interaccion, que interaccion realiza y si es completado si lo hace con exito
#Solo aparecen valores outliers
#Aqui apareceran jugadores que analizan el problema antes de ponerse a resolverlo
jugadoresOutliers_Inicio_Interaccion = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInicio_Interaccion_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Inicio_Interaccion, orient='index').transpose()

In [ ]:
#Tiempo entre una interaccion (inicio, reinicio, interaccionTarjeta) y completar el nivel (con exito o salir al menu), que accion hizo antes
#Solo aparecen valores outliers
#Aqui apareceran jugadores que revisan si lo que han hecho tiene sentido y va a funcionar, "sin probar a lo loco"
jugadoresOutliers_Interaccion_Completado = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInteraccion_Completado_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Interaccion_Completado, orient='index').transpose()

In [ ]:
#Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta, que accion hizo antes
#Solo aparecen valores outliers
#Aqui apareceran jugadores que piensan cada movimiento que hacen antes de hacerlo
jugadoresOutliers_Interaccion_Interaccion = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInteraccion_Interaccion_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Interaccion_Interaccion, orient='index').transpose()

In [ ]:
pd.DataFrame(result["anomalias"])
#Se ha interactuado en un nivel que no estaba iniciado -> Probablemente sea porque completa el nivel, minimiza la ventana de siguiente nivel o menu y se pone a mover las tarjetas

In [ ]:
def verTrazas(rawData):
    for e in rawData:
        name = e["actor"]["name"]
        if(name == "olugf"):
            print(e)
            print("--------")

JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)
verTrazas(rawData)
JSONFile.close()

In [ ]:
#Podemos ordenar tiempos gracias a la sobrecarga de operadores:
tiempos = [Tiempo("1h/30m/15s"), Tiempo("1h/30m"), Tiempo("1h"), Tiempo("30m/17s"), Tiempo("30m"), Tiempo("28m"), Tiempo("1m/13s"), Tiempo("45s")]
tiempos.sort()
print(tiempos)